<pre>
 ____   ____    _    ____  _____                          _      _     
|  _ \ / ___|  / \  / ___|| ____|     _ __ ___   ___   __| | ___| |___ 
| | | | |     / _ \ \___ \|  _| _____| '_ ` _ \ / _ \ / _` |/ _ \ / __|
| |_| | |___ / ___ \ ___) | |__|_____| | | | | | (_) | (_| |  __/ \__ \
|____/ \____/_/   \_\____/|_____|    |_| |_| |_|\___/ \__,_|\___|_|___/
                                                                        
</pre>

# DCASE-models Notebooks
Python Notebooks for [DCASE-models](https://github.com/pzinemanas/DCASE-models)

---

### About 
This Notebook reproduces the results for **Sound Event Detection (SED)** presented in:
<ul>
<li><a href="https://arxiv.org/pdf/1706.02291.pdf"><strong>
    Sound event detection using spatial features and convolutional recurrent neural network </strong></a>
   S. Adavanne, P. Pertilä, T. Virtanen ICASSP 2017.
    <br>
   <a type="button" class="btn btn-default btn-xs" target="_blank" href="https://arxiv.org/pdf/1706.02291.pdf"> PDF </a>
   <a type="button" class="btn btn-default btn-xs" target="_blank" href="https://ieeexplore.ieee.org/document/7952260"> IEEE</a>
    </li>   
</ul>

### Overview

This paper extends the Convolutional Recurrent Neural Network (CRNN) proposed by Cakir et al. [[C-CRNN]](https://ieeexplore.ieee.org/document/7933050) for multichannel audio event detection using spatial features (generalized cross-correlation with phase based weighting
weighting) and autocorrelation, besides log mel-band energy. The system is evaluated on TUT Sound Events 2009 ([TUT-SED 2009](http://www.cs.tut.fi/sgn/arg/taslp2017-crnn-sed/#tut-sed-2009)- not public)  TUT Sound Events 2016 Development set ([TUT-SED 2016](http://www.cs.tut.fi/sgn/arg/taslp2017-crnn-sed/#tut-sed-2016)).

We'll use TUT-SED 2017, and use mel band energies only.

### Organization

The Notebook is organized into the following sections.
* [1. Load parameters](#LoadParameters)
* [2. Extract features](#ExtractFeatures)
* [3. Load data](#LoadData)
* [4. Initialize model](#InitModel)
* [5. Train model](#TrainModel)
* [6. Evaluate model](#EvaluateModel)

In [2]:
%load_ext autoreload
%autoreload 2
rootdir_path = '../../../'
import sys
import os
import json
import warnings
import glob
import numpy as np
import argparse

sys.path.append(rootdir_path)
from dcase_models.util.files import load_json, mkdir_if_not_exists
from dcase_models.data.data_generator import DataGenerator
from dcase_models.data.datasets import TUTSoundEvents2017
from dcase_models.model.models import A_CRNN
from dcase_models.data.features import MelSpectrogram
from dcase_models.data.scaler import Scaler
from dcase_models.util.files import load_json
from dcase_models.util.data import evaluation_setup

os.environ["CUDA_VISIBLE_DEVICES"] = "1"

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


<a id="LoadeParameters"></a>
## 1. Load parameters

Dataset parameters are stored in a json file on the root directory.

In [3]:
# load all parameters from json file
params = load_json(os.path.join(rootdir_path, 'parameters.json'))
# set the dataset we are going to use
dataset = 'TUTSoundEvents2017'

# get dataset parameters
params_dataset = params["datasets"][dataset]

# get feature extraction parameters
params_features = params["features"]

# get training parameters
params_train = params["train"]

params_model = params["models"]["A_CRNN"]



Check that the values of the parameters are correct.

In [4]:
# print the dataset parameters 
print("Dataset Parameters:\n", json.dumps(params_dataset, indent=4, sort_keys=True))
# print feature extraction parameters 
print("Features' Parameters:\n",json.dumps(params_features, indent=4, sort_keys=True))
# print training parameters 
print("Training Parameters:\n",json.dumps(params_train, indent=4, sort_keys=True))

Dataset Parameters:
 {
    "dataset_path": "datasets/TUTSoundEvents2017",
    "evaluation_mode": "cross-validation-with-test"
}
Features' Parameters:
 {
    "MelSpectrogram": {
        "mel_bands": 64,
        "n_fft": 1024
    },
    "Openl3": {
        "content_type": "env",
        "embedding_size": 512,
        "input_repr": "mel256"
    },
    "Spectrogram": {
        "n_fft": 1024
    },
    "audio_hop": 680,
    "audio_win": 1024,
    "sequence_hop_time": 0.5,
    "sequence_time": 1.0,
    "sr": 22050
}
Training Parameters:
 {
    "batch_size": 1,
    "considered_improvement": 0,
    "early_stopping": 30,
    "epochs": 50,
    "learning_rate": 0.001,
    "optimizer": "Adam",
    "verbose": 1
}


<a id="ExtractFeatures"></a>
## 2. Extract features

Initialize Feature Extractor and Data Generator.

In [30]:
# Define params

params_features = {
    "MelSpectrogram": {
        "mel_bands": 40,
        "n_fft": 2048
    },
    "audio_hop": 882, # 20 ms
    "audio_win": 1764, # 40 ms
    "sequence_hop_time": 1.0,
    "sequence_time": 1.0,
    "sr": 44100
}
# Initialize Feature Extractor
features = MelSpectrogram(sequence_time=params_features['sequence_time'], 
                                   sequence_hop_time=params_features['sequence_hop_time'], 
                                   audio_win=params_features['audio_win'], 
                                   audio_hop=params_features['audio_hop'], 
                                   sr=params_features['sr'],
                                   **params_features['MelSpectrogram'])


print(features.get_shape())

(10, 50, 40)


In [31]:
# Initialize Data Generator as an instance of URBAN_SED
dataset = TUTSoundEvents2017(os.path.join(rootdir_path, params_dataset["dataset_path"]))

Check if dataset exists, and download it if doesn't exist.

In [32]:
dataset.download()

Extract the features (if they were not extracted before).

In [33]:
if not features.check_if_extracted(dataset):
    features.extract(dataset)
print('Done!')

[############################################################] 32/32
Done!


<a id="LoadData"></a>
## 3. Load data

In [34]:
# Get train/test folds
folds_train, folds_val, folds_test = evaluation_setup('fold1', dataset.fold_list,
                                             params_dataset['evaluation_mode'],
                                             use_validate_set=True)
#initialise Data Generator
data_gen_train = DataGenerator(dataset, features, folds=folds_train,
                               batch_size=params_train['batch_size'],
                               shuffle=True, train=True, scaler=None)

And also fit a scaler and transform the training data.

In [35]:
scaler = Scaler(normalizer=params_model['normalizer'])
print('Fitting scaler ...')
scaler.fit(data_gen_train)
print('Done!')

data_gen_train.set_scaler(scaler)

Fitting scaler ...
Done!


Initialise data generator

In [36]:
data_gen_val = DataGenerator(dataset, features, folds=folds_val,
                             batch_size=params_train['batch_size'],
                             shuffle=False, train=False, scaler=scaler)

<a id="InitModel"></a>
## 4. Initialize model

In [37]:
features_shape = features.get_shape()
n_frames_cnn = features_shape[1]
n_freq_cnn = features_shape[2]
n_classes = len(dataset.label_list)

metrics = ['sed']

model_container = A_CRNN(model=None, model_path=None, n_classes=n_classes, 
                         n_frames_cnn=n_frames_cnn, n_freq_cnn=n_freq_cnn,
                         metrics=metrics, final_activation='sigmoid',
                         bidirectional=True)

model_container.model.summary()

0 (?, 50, 40, 128)
1 (?, 50, 8, 128)
2 (?, 50, 4, 128)
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           (None, 50, 40)            0         
_________________________________________________________________
lambda (Lambda)              (None, 50, 40, 1)         0         
_________________________________________________________________
conv2d_16 (Conv2D)           (None, 50, 40, 128)       1280      
_________________________________________________________________
batch_normalization_16 (Batc (None, 50, 40, 128)       160       
_________________________________________________________________
activation_16 (Activation)   (None, 50, 40, 128)       0         
_________________________________________________________________
max_pooling2d_16 (MaxPooling (None, 50, 8, 128)        0         
_________________________________________________________________
dropout_21 (Dropout) 

<a id="TrainModel"></a>
## 5. Train model

In [39]:
X_train, Y_train = data_gen_train.get_data()

In [41]:
#define path to save weights and training log
exp_folder = './'
# Replacing default training parameters by paper parameters
params_train["epochs"] = 300
params_train["early_stopping"] = 50
params_train["batch_size"] = 32

model_container.train((X_train, Y_train), data_gen_val, weights_path=exp_folder, 
                      label_list=dataset.label_list,
                      sequence_time_sec=params_features['sequence_hop_time'],
                      **params_train)

Epoch 1/300
4063/4063 [==============================] - 22s 5ms/step - loss: 1.7863
F1 = 0.4836, ER = 1.6335 - Best val F1: 0.4836
                  (IMPROVEMENT, saving)

Epoch 2/300
4063/4063 [==============================] - 18s 4ms/step - loss: 1.7212
F1 = 0.5052, ER = 1.3518 - Best val F1: 0.5052
                  (IMPROVEMENT, saving)

Epoch 3/300
4063/4063 [==============================] - 18s 4ms/step - loss: 1.6874
F1 = 0.4754, ER = 1.8069 - Best val F1: 0.5052 (1)

Epoch 4/300
4063/4063 [==============================] - 18s 4ms/step - loss: 1.6273
F1 = 0.5225, ER = 1.1624 - Best val F1: 0.5225
                  (IMPROVEMENT, saving)

Epoch 5/300
4063/4063 [==============================] - 18s 4ms/step - loss: 1.5586
F1 = 0.4642, ER = 1.3948 - Best val F1: 0.5225 (3)

Epoch 6/300
4063/4063 [==============================] - 18s 4ms/step - loss: 1.5395
F1 = 0.4679, ER = 1.2927 - Best val F1: 0.5225 (3)

Epoch 7/300
4063/4063 [==============================] - 18s 4ms/step 

KeyboardInterrupt: 

<a id="EvaluateModel"></a>
## 6. Evaluate Model

In [46]:
# Load best_weights
model_container.load_model_weights(exp_folder)

data_gen_test = DataGenerator(dataset, features, folds=folds_test,
                              batch_size=params_train['batch_size'],
                              shuffle=False, train=False, scaler=scaler)

kwargs = {'sequence_time_sec': params_features['sequence_hop_time'],
          'metric_resolution_sec': 1.0}
results = model_container.evaluate(data_gen_test, label_list=dataset.label_list, **kwargs)

print(results[metrics[0]])

Segment based metrics
  Evaluated length                  : 1753.00 sec
  Evaluated files                   : 8 
  Segment length                    : 1.00 sec

  Overall metrics (micro-average)
  F-measure
    F-measure (F1)                  : 36.08 %
    Precision                       : 27.36 %
    Recall                          : 52.97 %
  Error rate
    Error rate (ER)                 : 1.52 
    Substitution rate               : 0.36 
    Deletion rate                   : 0.11 
    Insertion rate                  : 1.05 
  Accuracy
    Sensitivity                     : 52.97 %
    Specificity                     : 74.12 %
    Balanced accuracy               : 63.54 %
    Accuracy                        : 70.83 %

  Class-wise average metrics (macro-average)
  F-measure
    F-measure (F1)                  : 28.27 %
    Precision                       : 21.27 %
    Recall                          : 37.85 %
  Error rate
    Error rate (ER)                 : 2.47 
    Deletion rate 